In [1]:
import tensorflow as tf
from __future__ import print_function
from __future__ import division
import collections
import nltk
import numpy as np
from nltk.tokenize.casual import TweetTokenizer
from w266_common import utils, vocabulary
import time
import datetime
import os
import re
np.random.seed(266)

/home/saulgrimaldo1/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import csv
tokenizer = TweetTokenizer()
x_data = []
x_contexts = []
labels = []
sentences  = []
contexts = []
with open('merged_data_v4.csv', 'r') as csvfile:
    linereader = csv.reader(csvfile, delimiter = '|')
    for i, row in enumerate(linereader):
        if i == 0:
            continue
        sentence, context, sarcasm = row
        sentence = re.sub("RT @[^\s]+:", "retweet", sentence)
        sentences.append(sentence)
        contexts.append(context)
        x_tokens = utils.canonicalize_words(tokenizer.tokenize(sentence), hashtags = True)
        context_tokens = utils.canonicalize_words(tokenizer.tokenize(context), hashtags = True)
        x_data.append(x_tokens)
        x_contexts.append(context_tokens)
        labels.append(int(sarcasm))



shuffle_indices = np.random.permutation(np.arange(len(labels)))
train_split_idx = int(0.7 * len(labels))
test_split_idx  = int(0.9 * len(labels))

train_indices = shuffle_indices[:train_split_idx]
validation_indices = shuffle_indices[train_split_idx:test_split_idx]
test_indices = shuffle_indices[test_split_idx:]


train_sentences = np.array(x_data)[train_indices]
train_contexts = np.array(x_contexts)[train_indices]
train_labels= np.array(labels)[train_indices] 
validation_sentences = np.array(x_data)[validation_indices]
validation_labels = np.array(labels)[validation_indices]
validation_contexts = np.array(x_contexts)[validation_indices]
test_sentences = np.array(x_data)[test_indices]  
test_contexts =  np.array(x_contexts)[test_indices]
test_labels = np.array(labels)[test_indices]  

In [3]:
a = [2]*4
a[2] = 6
a

[2, 2, 6, 2]

In [4]:
def transform_labels(raw_label_set, size):
    label_set = []
    for label in raw_label_set:
        labels = [0] * size
        labels[label] = 1
        label_set.append(labels)
    return np.array(label_set)

expanded_train_labels = transform_labels(train_labels, 2)
expanded_validation_labels = transform_labels(validation_labels,2)
expanded_test_labels = transform_labels(test_labels,2)

In [5]:
a = [1,3,4,5,6,7,8,8,1,5,6,7]
a + ["<PADDING>"]*5

[1,
 3,
 4,
 5,
 6,
 7,
 8,
 8,
 1,
 5,
 6,
 7,
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>']

In [6]:
class PaddingAndTruncating:
    def __init__(self, max_len):
        self.max_len = max_len
    
    def pad_or_truncate(self, sentence):
        sen_len = len(sentence)
        paddings = self.max_len - sen_len
        if paddings >=0:
            return list(sentence) + ["<PADDING>"] * paddings
        return sentence[0:paddings]
        
PadAndTrunc = PaddingAndTruncating(10)
        
        
PadAndTrunc.pad_or_truncate(["the","angry","man","is","angry"])

['the',
 'angry',
 'man',
 'is',
 'angry',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>']

In [7]:
from tensorflow.contrib import learn
vocab_size = 5000

#max_len = max([len(sent) for sent  in train_sentences])
PadAndTrunc =PaddingAndTruncating(40)
train_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, train_sentences))
train_context_padded = list(map(PadAndTrunc.pad_or_truncate, train_contexts))
validation_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, validation_sentences))
validation_context_padded = list(map(PadAndTrunc.pad_or_truncate, validation_contexts))
test_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, test_sentences))
test_context_padded = list(map(PadAndTrunc.pad_or_truncate, test_contexts))

vocab = vocabulary.Vocabulary(utils.flatten(list(train_sentences_padded) + list(train_context_padded)), vocab_size)
train_s = np.array(list(map(vocab.words_to_ids, train_sentences_padded)))
train_c = np.array(list(map(vocab.words_to_ids, train_context_padded)))
validation_s = np.array(list(map(vocab.words_to_ids, validation_sentences_padded)))
validation_c = np.array(list(map(vocab.words_to_ids, validation_context_padded)))
test_s = np.array(list(map(vocab.words_to_ids, test_sentences_padded)))
test_c = np.array(list(map(vocab.words_to_ids, test_context_padded)))
train_s

array([[  12,  877,   22, ...,    3,    3,    3],
       [  12,   80,   41, ...,    3,    3,    3],
       [  93,  618,    2, ...,    3,    3,    3],
       ..., 
       [  12,    2,  182, ...,    3,    3,    3],
       [   2, 2646, 3847, ...,    3,    3,    3],
       [  12,   39,  983, ...,    3,    3,    3]])

In [8]:
i = 3
("conv-maxpool-%s" % i)

'conv-maxpool-3'

In [9]:
class TextCNN(object):

    def __init__(
      self, sequence_length, num_classes, vocab_size, 
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x1 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x1")
        self.input_x2 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x2")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars1 = tf.nn.embedding_lookup(self.W, self.input_x1)
            self.embedded_chars_expanded1 = tf.expand_dims(self.embedded_chars1, -1)
            self.embedded_chars2 = tf.nn.embedding_lookup(self.W, self.input_x2)
            self.embedded_chars_expanded2 = tf.expand_dims(self.embedded_chars2, -1)

        # Create a convolution + avgpool layer for each filter size
        pooled_outputs = []

        for i, filter_size in enumerate(filter_sizes):
            with tf.variable_scope("conv-avgpool-%s" % i) as scope:
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]

                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
              
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded1,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h1 = tf.nn.tanh(tf.nn.bias_add(conv, b), name="tanh1")
               
                # Maxpooling over the outputs
                pooled = tf.nn.avg_pool(
                    h1,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                
                pooled_outputs.append(pooled)
                #pooled_outputs.append(pooled)
                scope.reuse_variables()
                 
                
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded2,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h2 = tf.nn.tanh(tf.nn.bias_add(conv, b), name="tanh2")
                # Maxpooling over the outputs
                pooled2 = tf.nn.avg_pool(
                    h2,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled2)

        # Combine all the pooled features

        num_filters_total = num_filters * len(filter_sizes) * 2

        self.h_pool = tf.concat(pooled_outputs, 1)

        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])


        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
        
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
           
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            self.labels = tf.argmax(self.input_y, 1)
            TP = tf.count_nonzero(self.predictions * self.labels)
            TN = tf.count_nonzero((self.predictions - 1) * (self.labels - 1))
            FP = tf.count_nonzero(self.predictions * (self.labels - 1))
            FN = tf.count_nonzero((self.predictions - 1) * self.labels)
            self.correct_predictions = tf.equal(self.predictions, self.labels)
            self.precision = TP / (TP + FP)
            self.recall = TP / (TP + FN)
            self.f1_score = 2 * self.precision * self.recall / (self.precision + self.recall)
            self.accuracy = tf.reduce_mean(tf.cast(self.correct_predictions, "float"), name="accuracy")

In [10]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [11]:
learning_rate = 0.1

tf.flags.DEFINE_float("dev_sample_percentage", 0.75, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 40, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "1", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 40, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.4, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0, "L2 regularization lambda (default: 0.0)")

# Training parameters
batch_size = 200
tf.flags.DEFINE_integer("batch_size", batch_size, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 10, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 20, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================


with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():

        cnn = TextCNN(
            sequence_length=train_s.shape[1],
            num_classes=2,
            vocab_size=vocab_size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)
        
         # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(learning_rate)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        #vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch1, x_batch2, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x1: x_batch1,
              cnn.input_x2: x_batch2,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch1, x_batch2, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x1: x_batch1,
              cnn.input_x2: x_batch2,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            #r 
            step, summaries, loss, accuracy, recall, precision, f1 = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy,cnn.recall, cnn.precision, cnn.f1_score],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}, rec {:g}, pre {:g}, f1 {:g}".format(time_str, step, loss, accuracy, recall, precision, f1))# recall, precision, f1))
            if writer:
                writer.add_summary(summaries, step)
        
        def error_analysis(x_batch1, x_batch2, y_batch, writer=None):
            feed_dict = {
              cnn.input_x1: x_batch1,
              cnn.input_x2: x_batch2,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            #r 
            step, summaries, loss, accuracy, recall, precision, f1, correct, scores,predictions  = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy,cnn.recall, cnn.precision, cnn.f1_score, cnn.correct_predictions, cnn.scores, cnn.predictions],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}, rec {:g}, pre {:g}, f1 {:g}".format(time_str, step, loss, accuracy, recall, precision, f1))# recall, precision, f1))
            if writer:
                writer.add_summary(summaries, step)
            return correct, scores, predictions

        # Generate batches
        batches = batch_iter(
            list(zip(train_s, train_c,  expanded_train_labels)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch1, x_batch2, y_batch = zip(*batch)
            train_step(x_batch1, x_batch1, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(validation_s, validation_c, expanded_validation_labels, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))
print("\nTest Set:")
correct, logits, predictions = error_analysis(test_s, test_c, expanded_test_labels, writer=dev_summary_writer)
print("")



Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=200
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.75
DROPOUT_KEEP_PROB=0.4
EMBEDDING_DIM=40
EVALUATE_EVERY=20
FILTER_SIZES=1
L2_REG_LAMBDA=0
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.neg
NUM_CHECKPOINTS=5
NUM_EPOCHS=10
NUM_FILTERS=40
POSITIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.pos

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-avgpool-0/W:0/grad/hist is illegal; using conv-avgpool-0/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-avgpool-0/W:0/grad/sparsity is illegal; using conv-avgpool-0/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-avgpool-0/b:0/grad/hist is illegal; using conv-avgpool-0/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-avgpool-0/b:0/grad/

2018-04-22T20:13:14.796482: step 101, loss 0.358958, acc 0.87
2018-04-22T20:13:14.872468: step 102, loss 0.272576, acc 0.875
2018-04-22T20:13:14.952475: step 103, loss 0.488639, acc 0.84
2018-04-22T20:13:15.036443: step 104, loss 0.379693, acc 0.87
2018-04-22T20:13:15.116482: step 105, loss 0.316909, acc 0.88
2018-04-22T20:13:15.192462: step 106, loss 0.271103, acc 0.885
2018-04-22T20:13:15.271731: step 107, loss 0.39276, acc 0.865
2018-04-22T20:13:15.356475: step 108, loss 0.274302, acc 0.89
2018-04-22T20:13:15.428450: step 109, loss 0.37812, acc 0.85
2018-04-22T20:13:15.506340: step 110, loss 0.391465, acc 0.865
2018-04-22T20:13:15.588458: step 111, loss 0.40381, acc 0.885
2018-04-22T20:13:15.666693: step 112, loss 0.262221, acc 0.89
2018-04-22T20:13:15.748398: step 113, loss 0.247644, acc 0.91
2018-04-22T20:13:15.829038: step 114, loss 0.434305, acc 0.875
2018-04-22T20:13:15.909967: step 115, loss 0.291643, acc 0.88
2018-04-22T20:13:15.989761: step 116, loss 0.38604, acc 0.85
2018-0

2018-04-22T20:13:26.326866: step 220, loss 0.410722, acc 0.826296, rec 0.971408, pre 0.754986, f1 0.849631

2018-04-22T20:13:26.413047: step 221, loss 0.387872, acc 0.86
2018-04-22T20:13:26.492517: step 222, loss 0.257965, acc 0.91
2018-04-22T20:13:26.576517: step 223, loss 0.275406, acc 0.895
2018-04-22T20:13:26.652493: step 224, loss 0.348566, acc 0.89
2018-04-22T20:13:26.732599: step 225, loss 0.317488, acc 0.855
2018-04-22T20:13:26.815433: step 226, loss 0.365887, acc 0.865
2018-04-22T20:13:26.892510: step 227, loss 0.3531, acc 0.865
2018-04-22T20:13:26.972519: step 228, loss 0.286236, acc 0.865
2018-04-22T20:13:27.060503: step 229, loss 0.292478, acc 0.895
2018-04-22T20:13:27.137760: step 230, loss 0.406115, acc 0.86
2018-04-22T20:13:27.217266: step 231, loss 0.320368, acc 0.875
2018-04-22T20:13:27.298246: step 232, loss 0.445906, acc 0.87
2018-04-22T20:13:27.375435: step 233, loss 0.246671, acc 0.91
2018-04-22T20:13:27.456544: step 234, loss 0.381265, acc 0.86
2018-04-22T20:13:27

2018-04-22T20:13:37.903600: step 340, loss 0.485888, acc 0.744074, rec 0.953079, pre 0.674624, f1 0.790033

2018-04-22T20:13:37.992511: step 341, loss 0.217934, acc 0.905
2018-04-22T20:13:38.068513: step 342, loss 0.261298, acc 0.92
2018-04-22T20:13:38.156554: step 343, loss 0.337933, acc 0.895
2018-04-22T20:13:38.234050: step 344, loss 0.466395, acc 0.855
2018-04-22T20:13:38.309231: step 345, loss 0.23481, acc 0.92
2018-04-22T20:13:38.395751: step 346, loss 0.275848, acc 0.89
2018-04-22T20:13:38.473026: step 347, loss 0.423859, acc 0.84
2018-04-22T20:13:38.552505: step 348, loss 0.2281, acc 0.905
2018-04-22T20:13:38.630401: step 349, loss 0.306794, acc 0.905
2018-04-22T20:13:38.713659: step 350, loss 0.39391, acc 0.865
2018-04-22T20:13:38.792530: step 351, loss 0.385508, acc 0.855
2018-04-22T20:13:38.873262: step 352, loss 0.378045, acc 0.875
2018-04-22T20:13:38.956533: step 353, loss 0.383974, acc 0.84
2018-04-22T20:13:39.032519: step 354, loss 0.265593, acc 0.9
2018-04-22T20:13:39.1

2018-04-22T20:13:44.825164: step 460, loss 0.793958, acc 0.594074, rec 0.997801, pre 0.554605, f1 0.712939

2018-04-22T20:13:44.866564: step 461, loss 0.304632, acc 0.875
2018-04-22T20:13:44.903101: step 462, loss 0.298867, acc 0.875
2018-04-22T20:13:44.939953: step 463, loss 0.346685, acc 0.895
2018-04-22T20:13:44.976517: step 464, loss 0.339781, acc 0.85
2018-04-22T20:13:45.013619: step 465, loss 0.264422, acc 0.9
2018-04-22T20:13:45.054454: step 466, loss 0.324825, acc 0.89
2018-04-22T20:13:45.092088: step 467, loss 0.238773, acc 0.925
2018-04-22T20:13:45.129153: step 468, loss 0.444983, acc 0.855
2018-04-22T20:13:45.165856: step 469, loss 0.387745, acc 0.86
2018-04-22T20:13:45.202556: step 470, loss 0.411659, acc 0.84
2018-04-22T20:13:45.240176: step 471, loss 0.287369, acc 0.87
2018-04-22T20:13:45.281847: step 472, loss 0.250736, acc 0.91
2018-04-22T20:13:45.319396: step 473, loss 0.3068, acc 0.85
2018-04-22T20:13:45.358794: step 474, loss 0.236286, acc 0.92
2018-04-22T20:13:45.39

In [12]:
wrong = correct == False
def incorrect_confidence(wrong, logits, predictions):
    indeces = np.where(wrong)
    wrong_predictions = predictions[indeces]
    wrong_logits = logits[indeces]
    
    return [[wrong_logits[i][value] - wrong_logits[i][1-value], indeces[0][i]] for i, value in enumerate(wrong_predictions)]


sorted(incorrect_confidence(wrong, logits, predictions), key = lambda logit: -logit[0])

[[3.8800895, 798],
 [3.6586144, 878],
 [2.7627788, 739],
 [2.4937358, 1347],
 [2.2221501, 1196],
 [2.2016802, 1186],
 [2.0854609, 462],
 [2.038799, 748],
 [2.0335665, 1280],
 [2.0248444, 1134],
 [1.9460719, 572],
 [1.7840009, 1052],
 [1.7175657, 926],
 [1.691916, 487],
 [1.6451406, 293],
 [1.6396594, 211],
 [1.5169232, 939],
 [1.4470859, 956],
 [1.3997362, 263],
 [1.325048, 1146],
 [1.3107753, 150],
 [1.2799933, 34],
 [1.264863, 84],
 [1.2616477, 936],
 [1.2471952, 174],
 [1.2355683, 1062],
 [1.2076569, 308],
 [1.1605282, 398],
 [1.1555448, 479],
 [1.1374881, 508],
 [1.1240113, 1148],
 [1.1211736, 454],
 [1.1208985, 191],
 [1.1124182, 70],
 [1.1031786, 952],
 [1.0616415, 505],
 [1.0329688, 637],
 [1.0207822, 286],
 [0.99851751, 291],
 [0.99112976, 675],
 [0.94670254, 382],
 [0.93777567, 835],
 [0.9345172, 955],
 [0.92688459, 1239],
 [0.90270978, 692],
 [0.88538486, 1092],
 [0.87993592, 605],
 [0.87174743, 707],
 [0.86846215, 894],
 [0.85658669, 638],
 [0.85612482, 229],
 [0.82828426, 1

In [13]:

print(test_sentences[878])
print(test_contexts[878])
print(test_labels[878])
print(predictions[878])
#definitely sarcastic

['what', 'is', 'with', 'that', '?', 'theyre', 'only', 'going', 'to', 'blame', 'the', 'guns', 'now', '?', 'no', 'people', '?', 'at', 'all', '?', 'oh', 'yeah', 'they', 'will', 'blame', 'trump', '.', 'there', 'is', 'no', 'agenda', 'though', '...', 'LINK']
['nan']
0
1


In [14]:

print(test_sentences[293])
print(test_contexts[293])
print(test_labels[293])
print(predictions[293])

# hard to say what's going on here. seems to be slightly sarcastic, not really sure though. Throwing me off too

['when', 'history', 'isn', '’', 't', 'being', 'altered', 'by', 'federal', 'govt', 'education', 'how', 'do', 'i', 'know', 'i', 'heard', 'mrs', 'obama', 'say', 'it', 'right', 'out', 'of', 'her', 'mouth', 'using', 'the', 'word', '(', '(', '(', 'HASHTAG', 'HASHTAG', ')', ')', ')', 'main', 'while', 'in', 'history', 'this', 'was', 'taught', 'HASHTAG', 'LINK']
['nan']
0
1


In [15]:
print(test_sentences[798])
print(test_contexts[798])
print(test_labels[798])
print(predictions[798])
# heavy use of hashtags is throwing this one off too

['LINK', '…', '…', '…', '…', '…', '…', '…', '…', '…', '…', '…', '…', '…', '…', '…', '…', '…', '…', '…', '…', 'once', 'upon', 'a', 'time', '(', 'spin', ')', 'featuring', 'ACCOUNT', 'of', 'd12', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG']
['nan']
1
0


In [16]:
print(test_sentences[1347])
print(test_contexts[1347])
print(test_labels[1347])
print(predictions[1347])
# emoji is throwing it off

['retweet', 'not', 'sure', 'about', 'the', 'cricket', 'ball', ',', 'but', 'bancroft', '’', 's', 'balls', 'must', 'be', 'reverse-swinging', 'quiet', 'a', 'bit', 'after', 'yesterday', '’', 's', 'events', '.', '😂', '😂', '😂', '…']
['nan']
0
1


In [17]:
print(test_sentences[1280])
print(test_contexts[1280])
print(test_labels[1280])
print(predictions[1280])

# heavy use of hashtags is throwing this one off. Majority of tweets with heavy hashtag usage (as decribed in EDA)
# are non-sarcatic

['a', 'thought-provoking', 'image', 'of', 'the', 'day', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'HASHTAG', 'LINK']
['nan']
1
0
